In [ ]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [ ]:
data_sample = load_dataset('QuyenAnhDE/Diseases_Symptoms')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
updated_data = [{'Name': item['Name'], 'Symptoms': item['Symptoms']} for item in data_sample['train']]

In [ ]:
df = pd.DataFrame(updated_data)

In [ ]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  try:
    device = torch.device('mps')
  except:
    device = torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
BATCH_SIZE = 8

In [ ]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [ ]:
# Dataset prep
class LanguageDataset(Dataset):
  def __init__(self, df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient='records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f'{x} | {y}'
    tokens = self.tokenizer.encode_plus(text, return_tensors = 'pt', max_length=128, padding='max_length', truncation=True)
    return tokens

  def fittest_max_length(self, df):
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x = 2
    while x < max_length: x = x * 2
    return x

In [ ]:
data_sample = LanguageDataset(df, tokenizer)

In [ ]:
data_sample

In [ ]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size
train_data, val_data = random_split(data_sample, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [ ]:
num_epochs = 2

In [ ]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [ ]:
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  epoch_training_loss = 0
  train_iterator = tqdm(train_loader, desc=f'Training epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}')
  for batch in train_iterator:
    optimizer.zero_grad()
    inputs = batch['input_ids'].squeeze(1).to(device)
    targets = inputs.clone()
    outputs = model(input_ids = inputs, labels=targets)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training Loss': loss.item()})
    epoch_training_loss += loss.item()
  avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

  model.eval()
  epoch_validation_loss = 0
  total_loss = 0
  valid_iterator = tqdm(val_loader, desc=f'Validation epoch {epoch+1}/{num_epochs}')
  with torch.no_grad():
    for batch in valid_iterator:
      inputs = batch['input_ids'].squeeze(1).to(device)
      targets = inputs.clone()
      outputs = model(input_ids = inputs, labels=targets)
      loss = outputs.loss
      total_loss += loss
      valid_iterator.set_postfix({'Validation Loss': loss.item()})
      epoch_validation_loss += loss.item()
    avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)

    end_time = time.time()
    epoch_duration_sec = end_time - start_time

    new_row = {
        'transformer': model_name,
        'batch_size': batch_size,
        'gpu': gpu,
        'epoch': epoch+1,
        'training_loss': avg_epoch_training_loss,
        'validation_loss': avg_epoch_validation_loss,
        'epoch_duration_sec': epoch_duration_sec
    }

    results.loc[len(results)] = new_row
    print(f'Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}')


Training epoch 1/2 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:10<00:00,  3.93it/s, Training Loss=0.65]
Validation epoch 1/2: 100%|██████████| 10/10 [00:00<00:00, 17.10it/s, Validation Loss=0.446]


Epoch: 1, Validation Loss: 0.6893932223320007


Training epoch 2/2 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.00it/s, Training Loss=0.442]
Validation epoch 2/2: 100%|██████████| 10/10 [00:00<00:00, 17.42it/s, Validation Loss=0.441]

Epoch: 2, Validation Loss: 0.6788629293441772


In [ ]:
### If don't want to keep records


# for epoch in range(num_epochs):
#     start_time = time.time()
#     model.train()
#     epoch_training_loss = 0

#     # Training loop
#     train_iterator = tqdm(train_loader, desc=f'Training epoch {epoch+1}/{num_epochs}')
#     for batch in train_iterator:
#         optimizer.zero_grad()
#         inputs = batch['input_ids'].squeeze(1).to(device)
#         targets = inputs.clone()
#         outputs = model(input_ids=inputs, labels=targets)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         train_iterator.set_postfix({'Training Loss': loss.item()})
#         epoch_training_loss += loss.item()
#     avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

#     # Validation loop
#     model.eval()
#     epoch_validation_loss = 0
#     valid_iterator = tqdm(val_loader, desc=f'Validation epoch {epoch+1}/{num_epochs}')
#     with torch.no_grad():
#         for batch in valid_iterator:
#             inputs = batch['input_ids'].squeeze(1).to(device)
#             targets = inputs.clone()
#             outputs = model(input_ids=inputs, labels=targets)
#             loss = outputs.loss
#             valid_iterator.set_postfix({'Validation Loss': loss.item()})
#             epoch_validation_loss += loss.item()
#     avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)

#     # Print epoch summary
#     epoch_duration_sec = time.time() - start_time
#     print(f"Epoch {epoch+1}/{num_epochs} | Training Loss: {avg_epoch_training_loss:.4f} | Validation Loss: {avg_epoch_validation_loss:.4f} | Time: {epoch_duration_sec:.2f}s")


In [ ]:
input_str = 'Heart Attack'

In [ ]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [ ]:
input_ids

tensor([[28541,  8307]], device='cuda:0')

In [ ]:
output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
output

tensor([[28541,  8307,   930, 27419,  3867,    11,  5801, 36051, 50256]],
       device='cuda:0')

In [ ]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
decoded_output

'Heart Attack | Difficulty moving, rapid heartbeat'

In [ ]:
torch.save(model, 'SmallDiseaseM.pt')